In [12]:
""" Create shapefile and csv table for hybas, fao and upstream/downstream
-------------------------------------------------------------------------------

This script needs a revision or needs to be archived. The database is based
on stroing lists and is a non normalized version. violates N-1.


Author: Rutger Hofste
Date: 20170829
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:
    TESTING (Boolean) : Toggle testing case.
    SCRIPT_NAME (string) : Script name.
    OUTPUT_VERSION (integer) : output version.
    DATABASE_ENDPOINT (string) : RDS or postGreSQL endpoint.
    DATABASE_NAME (string) : Database name.
    TABLE_NAME_AREA_30SPFAF06 (string) : Table name used for areas. Must exist
        on same database as used in rest of script.
    S3_INPUT_PATH_RIVERDISCHARGE (string) : AWS S3 input path for 
        riverdischarge.    
    S3_INPUT_PATH_DEMAND (string) : AWS S3 input path for 
        demand.    

"""

S3_INPUT_PATH_FAO ="s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output_V07/"
S3_INPUT_PATH_DOWNSTREAM = "s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output_V02/"
S3_INPUT_PATH_HYBAS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V02/output_V04/"

SCRIPT_NAME = "Y2017M08D29_RH_Merge_FAONames_Upstream_V01"
OUTPUT_VERSION = 3

INPUT_FILE_NAME_FAO = "hybas_lev06_v1c_merged_fiona_withFAO_V07.pkl"
INPUT_FILE_NAME_DOWNSTREAM = "hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.pkl"
INPUT_FILE_NAME_HYBAS = "hybas_lev06_v1c_merged_fiona_V04.shp"

OUTPUT_FILE_NAME = "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V%0.2d" %(OUTPUT_VERSION)

ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input ec2: " + ec2_input_path,
      "\nOutput ec2: " + ec2_output_path,
      "\nInput s3 FAO: " + S3_INPUT_PATH_FAO,
      "\nInput s3 Downstream: " + S3_INPUT_PATH_DOWNSTREAM,
      "\nInput s3 Hybas: " + S3_INPUT_PATH_HYBAS,
      "\nOutput s3: " + s3_output_path)



Input ec2: /volumes/data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03 
Output ec2: /volumes/data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output_V03 
Input s3 FAO: s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output_V07/ 
Input s3 Downstream: s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output_V02/ 
Input s3 Hybas: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V02/output_V04/ 
Output s3: s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output_V03/


In [5]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M05D24 UTC 11:48


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [6]:
!rm -r {ec2_input_path}
!rm -r {ec2_output_path}

!mkdir -p {ec2_input_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03': No such file or directory
rm: cannot remove '/volumes/data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output_V03': No such file or directory


In [7]:
!aws s3 cp {S3_INPUT_PATH_FAO} {ec2_input_path} --recursive 

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output_V07/hybas_lev06_v1c_merged_fiona_withFAO_V07_link.pkl to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev06_v1c_merged_fiona_withFAO_V07_link.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output_V07/hybas_lev06_v1c_merged_fiona_withFAO_V07.pkl to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev06_v1c_merged_fiona_withFAO_V07.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output_V07/hybas_lev06_v1c_merged_fiona_withFAO_V07_link.csv to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev06_v1c_merged_fiona_withFAO_V07_link.csv
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output_V07/hybas_lev06_v1c_merged_fiona_withFAO_V07.csv to ../../../../data/Y2017M08D29_RH

In [8]:
!aws s3 cp {S3_INPUT_PATH_DOWNSTREAM} {ec2_input_path} --recursive 

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output_V02/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output_V02/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.pkl to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output_V02/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01 to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01


In [9]:
!aws s3 cp {S3_INPUT_PATH_HYBAS} {ec2_input_path} --recursive --exclude *.tif

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V02/output_V04/hybas_lev06_v1c_merged_fiona_V04.cpg to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev06_v1c_merged_fiona_V04.cpg
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V02/output_V04/hybas_lev00_v1c_merged_fiona_V04.cpg to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev00_v1c_merged_fiona_V04.cpg
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V02/output_V04/hybas_lev00_v1c_merged_fiona_V04.prj to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev00_v1c_merged_fiona_V04.prj
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V02/output_V04/hybas_lev06_v1c_merged_fiona_V04.prj to ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input_V03/hybas_lev06_v1c_merged_fiona_V04.prj
download: s3://w

In [10]:
import os
if 'GDAL_DATA' not in os.environ:
    os.environ['GDAL_DATA'] = r'/usr/share/gdal/2.1'
from osgeo import gdal,ogr,osr
'GDAL_DATA' in os.environ
# If false, the GDAL_DATA variable is set incorrectly. You need this variable to obtain the spatial reference
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import time

In [14]:
dfFAO = pd.read_pickle(os.path.join(ec2_input_path,INPUT_FILE_NAME_FAO))


In [17]:
dfFAO.head()

,SUB_NAME,MAJ_NAME,FAOid_copy
PFAF_ID,,,
111011,[Wadi El Naqa],"[Africa, Red Sea - Gulf of Aden Coast]",[MAJ_BAS_7019_SUB_BAS_0190312]
111012,[Egyptian east coast],"[Africa, Red Sea - Gulf of Aden Coast]",[MAJ_BAS_7019_SUB_BAS_0190313]
111013,[Egyptian east coast],"[Africa, Red Sea - Gulf of Aden Coast]",[MAJ_BAS_7019_SUB_BAS_0190313]
111014,[Egyptian east coast],"[Africa, Red Sea - Gulf of Aden Coast]",[MAJ_BAS_7019_SUB_BAS_0190313]
111015,[Egyptian east coast],"[Africa, Red Sea - Gulf of Aden Coast]",[MAJ_BAS_7019_SUB_BAS_0190313]


In [18]:
dfDownstream = pd.read_pickle(os.path.join(ec2_input_path,INPUT_FILE_NAME_DOWNSTREAM))

In [ ]:
dfDownstream = dfDownstream.set_index("PFAF_ID", drop=False)

In [19]:
dfDownstream.head()

,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,Upstream_HYBAS_IDs,Upstream_PFAF_IDs,Downstream_HYBAS_IDs,Downstream_PFAF_IDs,NEXT_SINK_PFAF
HYBAS_ID2,,,,,,,,,,,,,,,,,,,
6060000010,0,6060000010,0,6060000010,6060000010,0.0,0.0,4317.4,4317.4,611001,0,1,0,1,[],[],[],[],611001.0
6060000200,1,6060000200,0,6060000200,6060000200,0.0,0.0,35995.5,35996.7,611002,0,0,1,2,[],[],[],[],611002.0
6060000210,2,6060000210,0,6060000210,6060000210,0.0,0.0,443.9,443.9,611003,0,1,0,3,[],[],[],[],611003.0
6060000240,3,6060000240,0,6060000240,6060000240,0.0,0.0,2186.3,2186.3,611004,0,0,1,4,[],[],[],[],611004.0
6060000250,4,6060000250,0,6060000250,6060000250,0.0,0.0,6533.8,6533.8,611005,0,1,0,5,[],[],[],[],611005.0


In [20]:
dfDownstream.drop("Unnamed: 0",1,inplace=True)

In [21]:
gdfHybas = gpd.read_file(os.path.join(ec2_input_path,INPUT_FILE_NAME_HYBAS))
gdfHybas = gdfHybas.set_index("PFAF_ID", drop=False)

In [22]:
dfHybas = pd.DataFrame(gdfHybas["PFAF_ID"])

Merging the the downstream and FAO datasets, adding Hybas geometry and export both Excel sheet and dataset.

In [23]:
dfOut = dfDownstream.merge(dfFAO,how="outer")

MergeError: No common columns to perform merge on

In [16]:
dfOut = dfOut.set_index("PFAF_ID",drop=False)

In [17]:
gdfHybas.dtypes

HYBAS_ID       int64
NEXT_DOWN      int64
NEXT_SINK      int64
MAIN_BAS       int64
DIST_SINK    float64
DIST_MAIN    float64
SUB_AREA     float64
UP_AREA      float64
PFAF_ID        int64
ENDO           int64
COAST          int64
ORDER          int64
SORT           int64
geometry      object
dtype: object

In [18]:
gdfHybasSimple = gpd.GeoDataFrame(dfHybas, geometry=gdfHybas.geometry)

In [19]:
gdfHybasSimple.to_file(os.path.join(ec2_output_path,OUTPUT_FILE_NAME+".shp"))

In [20]:
dfOut.to_csv(os.path.join(ec2_output_path,OUTPUT_FILE_NAME+".csv"))

In [21]:
dfOut.to_pickle(os.path.join(ec2_output_path,OUTPUT_FILE_NAME+".pkl"))

In [22]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive

upload: ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V03.prj to s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V03.prj
upload: ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V03.cpg to s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V03.cpg
upload: ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V03.shx to s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V03.shx
upload: ../../../../data/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO

In [23]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:02:27.045097


Previous Runs:  
0:02:27.045097